# Data Wrangling Project - Dog Rating

In this project I'll utilizating the knowledge I learned in previious session to gathering, assessing ,cleaning and analysing the dog rating dataset

In [2]:
import pandas as pd
import numpy as np
import requests
import tweepy
import json
import timeit
import re


In [3]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, retry_count=3, retry_delay=5,
                 retry_errors=set([401, 404, 500, 503]),wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

## Gathering Data

In [4]:
dogg=pd.read_csv('twitter-archive-enhanced.csv')

In [145]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
response

with open((url.split('/')[-1]), mode='wb') as file:
    file.write(response.content)

In [5]:
pred=pd.read_csv('image-predictions.tsv',sep='\t')

In [6]:
dogg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [7]:
pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


Take a look at both dataset, in the twitter archive table have more data than the prediction (2356 vs 2075), I'll using the tweet_id from this one to fetch my json data from Tweepy API

In [8]:
status_list=[]

In [47]:
tweet_id=dogg['tweet_id'].tolist()

In [78]:
for tweet in tweet_id:
    try:
        status=api.get_status(tweet, tweet_mode='extended')._json
        status_list.append(status)
    except tweepy.TweepError as e:
        print(e)
        print(tweet)
        pass


[{'code': 144, 'message': 'No status found with that ID.'}]
888202515573088257
[{'code': 144, 'message': 'No status found with that ID.'}]
873697596434513921
[{'code': 144, 'message': 'No status found with that ID.'}]
869988702071779329
[{'code': 144, 'message': 'No status found with that ID.'}]
866816280283807744
[{'code': 144, 'message': 'No status found with that ID.'}]
861769973181624320
[{'code': 144, 'message': 'No status found with that ID.'}]
845459076796616705
[{'code': 144, 'message': 'No status found with that ID.'}]
842892208864923648
[{'code': 144, 'message': 'No status found with that ID.'}]
837012587749474308
[{'code': 144, 'message': 'No status found with that ID.'}]
827228250799742977
[{'code': 144, 'message': 'No status found with that ID.'}]
802247111496568832
Rate limit reached. Sleeping for: 291
[{'code': 144, 'message': 'No status found with that ID.'}]
775096608509886464
Rate limit reached. Sleeping for: 706


In [79]:
len(status_list)

2517

In [182]:

with open('tweet_json.txt', 'w') as outfile:  
    json.dump([status for status in status_list],outfile)

Now we have saved all the tweet status data into a json file for the future.

In [9]:
with open('tweet_json.txt') as json_file:  
    data = json.load(json_file)

Creating a dataframe with tweet id, retweet count and favorites count

In [10]:
count=pd.DataFrame(columns=['tweet_id','ret#','fav#'],index=range(len(data))) 

In [11]:
for i in range(len(data)):
    count.loc[i]['tweet_id']=int(data[i]['id_str'])
    count.loc[i]['ret#']=data[i]['retweet_count']
    count.loc[i]['fav#']=data[i]['favorite_count']


In [12]:
count['tweet_id']=count['tweet_id'].astype(str).astype(int)

## Assess Data

In [13]:
dogg.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [14]:
dogg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [15]:
all_columns = pd.Series(list(dogg) + list(pred) + list(count))

In [16]:
all_columns[all_columns.duplicated()]

17    tweet_id
29    tweet_id
dtype: object

In [17]:
list(dogg)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'retweeted_status_id',
 'retweeted_status_user_id',
 'retweeted_status_timestamp',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo']

In [18]:
dogg[dogg['expanded_urls'].isnull()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
30,886267009285017600,8.862664e+17,2.281182e+09,2017-07-15 16:51:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@NonWhiteHat @MayhewMayhem omg hello tanner yo...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
55,881633300179243008,8.816070e+17,4.738443e+07,2017-07-02 21:58:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@roushfenway These are good dogs but 17/10 is ...,NaN,NaN,NaN,NaN,17,10,None,None,None,None,None
64,879674319642796034,8.795538e+17,3.105441e+09,2017-06-27 12:14:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@RealKentMurphy 14/10 confirmed,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
113,870726314365509632,8.707262e+17,1.648776e+07,2017-06-02 19:38:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@ComplicitOwl @ShopWeRateDogs &gt;10/10 is res...,NaN,NaN,NaN,NaN,10,10,None,None,None,None,None
148,863427515083354112,8.634256e+17,7.759620e+07,2017-05-13 16:15:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@Jack_Septic_Eye I'd need a few more pics to p...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
179,857214891891077121,8.571567e+17,1.806710e+08,2017-04-26 12:48:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@Marc_IRL pixelated af 12/10,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
185,856330835276025856,NaN,NaN,2017-04-24 02:15:55 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @Jenna_Marbles: @dog_rates Thanks for ratin...,8.563302e+17,66699013.0,2017-04-24 02:13:14 +0000,NaN,14,10,None,None,None,None,None
186,856288084350160898,8.562860e+17,2.792810e+08,2017-04-23 23:26:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@xianmcguire @Jenna_Marbles Kardashians wouldn...,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
188,855862651834028034,8.558616e+17,1.943518e+08,2017-04-22 19:15:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@dhmontgomery We also gave snoop dogg a 420/10...,NaN,NaN,NaN,NaN,420,10,None,None,None,None,None
189,855860136149123072,8.558585e+17,1.361572e+07,2017-04-22 19:05:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@s8n You tried very hard to portray this good ...,NaN,NaN,NaN,NaN,666,10,None,None,None,None,None


In [19]:
dogg['expanded_urls'].value_counts()

https://vine.co/v/ea0OwvPTx9l                                                                                                                                                                                                                                                                                2
https://twitter.com/dog_rates/status/866334964761202691/photo/1,https://twitter.com/dog_rates/status/866334964761202691/photo/1                                                                                                                                                                              2
https://twitter.com/dog_rates/status/783839966405230592/photo/1,https://twitter.com/dog_rates/status/783839966405230592/photo/1,https://twitter.com/dog_rates/status/783839966405230592/photo/1                                                                                                              2
https://twitter.com/dog_rates/status/820749716845686786/photo/1,https://twitter.com/dog_rat

In [20]:
dogg.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [21]:
dogg[dogg['rating_denominator']!=10]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
313,835246439529840640,8.352460e+17,2.625958e+07,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@jonnysun @Lin_Manuel ok jomny I know you're e...,NaN,NaN,NaN,NaN,960,0,None,None,None,None,None
342,832088576586297345,8.320875e+17,3.058208e+07,2017-02-16 04:45:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@docmisterio account started on 11/15/15,NaN,NaN,NaN,NaN,11,15,None,None,None,None,None
433,820690176645140481,NaN,NaN,2017-01-15 17:52:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",The floofs have been released I repeat the flo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/820690176...,84,70,None,None,None,None,None
516,810984652412424192,NaN,NaN,2016-12-19 23:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Sam. She smiles 24/7 &amp; secretly aspir...,NaN,NaN,NaN,"https://www.gofundme.com/sams-smile,https://tw...",24,7,Sam,None,None,None,None
784,775096608509886464,NaN,NaN,2016-09-11 22:20:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: After so many requests, this is...",7.403732e+17,4.196984e+09,2016-06-08 02:41:38 +0000,https://twitter.com/dog_rates/status/740373189...,9,11,None,None,None,None,None
902,758467244762497024,NaN,NaN,2016-07-28 01:00:57 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Why does this never happen at my front door......,NaN,NaN,NaN,https://twitter.com/dog_rates/status/758467244...,165,150,None,None,None,None,None
1068,740373189193256964,NaN,NaN,2016-06-08 02:41:38 +0000,"<a href=""http://twitter.com/download/iphone"" r...","After so many requests, this is Bretagne. She ...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/740373189...,9,11,None,None,None,None,None
1120,731156023742988288,NaN,NaN,2016-05-13 16:15:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to this unbelievably well behaved sq...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/731156023...,204,170,this,None,None,None,None
1165,722974582966214656,NaN,NaN,2016-04-21 02:25:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Happy 4/20 from the squad! 13/10 for all https...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/722974582...,4,20,None,None,None,None,None
1202,716439118184652801,NaN,NaN,2016-04-03 01:36:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bluebert. He just saw that both #Final...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/716439118...,50,50,Bluebert,None,None,None,None


In [22]:
dogg[dogg['rating_numerator']==0]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
315,835152434251116546,NaN,NaN,2017-02-24 15:40:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you're so blinded by your systematic plag...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/835152434...,0,10,None,None,None,None,None
1016,746906459439529985,7.468859e+17,4.196984e+09,2016-06-26 03:22:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...","PUPDATE: can't see any. Even if I could, I cou...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/746906459...,0,10,None,None,None,None,None


In [23]:
dogg.source.value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

In [24]:
dogg.name.value_counts()

None         745
a             55
Charlie       12
Cooper        11
Oliver        11
Lucy          11
Tucker        10
Penny         10
Lola          10
Winston        9
Bo             9
Sadie          8
the            8
an             7
Buddy          7
Toby           7
Bailey         7
Daisy          7
Leo            6
Jax            6
Rusty          6
Bella          6
Koda           6
Oscar          6
Stanley        6
Dave           6
Jack           6
Milo           6
Scout          6
Sammy          5
            ... 
Luther         1
Pavlov         1
Carper         1
Coopson        1
Danny          1
Harvey         1
Lulu           1
DonDon         1
Bobby          1
Skittles       1
Dunkin         1
Doobert        1
Kallie         1
Ralphson       1
Finnegus       1
Chuck          1
Bruno          1
Snicku         1
Butters        1
Sprinkles      1
Darrel         1
Billy          1
Nida           1
Gordon         1
Bones          1
Severus        1
Jim            1
Rey           

In [25]:
(dogg['doggo']+dogg['floofer']+dogg['pupper']+dogg['puppo']).value_counts()

NoneNoneNoneNone        1976
NoneNonepupperNone       245
doggoNoneNoneNone         83
NoneNoneNonepuppo         29
doggoNonepupperNone       12
NoneflooferNoneNone        9
doggoNoneNonepuppo         1
doggoflooferNoneNone       1
dtype: int64

In [26]:
dogg[(dogg["doggo"]=='doggo') & (dogg["pupper"] == 'pupper')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
460,817777686764523521,NaN,NaN,2017-01-07 16:59:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...","This is Dido. She's playing the lead role in ""...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/817777686...,13,10,Dido,doggo,None,pupper,None
531,808106460588765185,NaN,NaN,2016-12-12 00:29:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have Burke (pupper) and Dexter (doggo)...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/808106460...,12,10,None,doggo,None,pupper,None
565,802265048156610565,7.331095e+17,4.196984e+09,2016-11-25 21:37:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...","Like doggo, like pupper version 2. Both 11/10 ...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/802265048...,11,10,None,doggo,None,pupper,None
575,801115127852503040,NaN,NaN,2016-11-22 17:28:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bones. He's being haunted by another d...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/801115127...,12,10,Bones,doggo,None,pupper,None
705,785639753186217984,NaN,NaN,2016-10-11 00:34:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Pinot. He's a sophisticated doggo. You...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/785639753...,10,10,Pinot,doggo,None,pupper,None
733,781308096455073793,NaN,NaN,2016-09-29 01:42:20 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...","Pupper butt 1, Doggo 0. Both 12/10 https://t.c...",NaN,NaN,NaN,https://vine.co/v/5rgu2Law2ut,12,10,None,doggo,None,pupper,None
778,775898661951791106,NaN,NaN,2016-09-14 03:27:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: Like father (doggo), like son (...",7.331095e+17,4.196984e+09,2016-05-19 01:38:16 +0000,https://twitter.com/dog_rates/status/733109485...,12,10,None,doggo,None,pupper,None
822,770093767776997377,NaN,NaN,2016-08-29 03:00:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is just downright precious...,7.410673e+17,4.196984e+09,2016-06-10 00:39:48 +0000,https://twitter.com/dog_rates/status/741067306...,12,10,just,doggo,None,pupper,None
889,759793422261743616,NaN,NaN,2016-07-31 16:50:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...","Meet Maggie &amp; Lila. Maggie is the doggo, L...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/759793422...,12,10,Maggie,doggo,None,pupper,None
956,751583847268179968,NaN,NaN,2016-07-09 01:08:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Please stop sending it pictures that don't eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/751583847...,5,10,None,doggo,None,pupper,None


In [27]:
dogg[(dogg["doggo"]=='doggo') & (dogg["puppo"] == 'puppo')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
191,855851453814013952,NaN,NaN,2017-04-22 18:31:02 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a puppo participating in the #ScienceMa...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/855851453...,13,10,None,doggo,None,None,puppo


In [28]:
dogg[(dogg["doggo"]=='doggo') & (dogg["floofer"] == 'floofer')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
200,854010172552949760,NaN,NaN,2017-04-17 16:34:26 +0000,"<a href=""http://twitter.com/download/iphone"" r...","At first I thought this was a shy doggo, but i...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/854010172...,11,10,None,doggo,floofer,None,None


In [29]:
pred.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [30]:
pred.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [31]:
pred.p1.value_counts()

golden_retriever             150
Labrador_retriever           100
Pembroke                      89
Chihuahua                     83
pug                           57
chow                          44
Samoyed                       43
toy_poodle                    39
Pomeranian                    38
cocker_spaniel                30
malamute                      30
French_bulldog                26
miniature_pinscher            23
Chesapeake_Bay_retriever      23
seat_belt                     22
German_shepherd               20
Staffordshire_bullterrier     20
Siberian_husky                20
web_site                      19
Cardigan                      19
Eskimo_dog                    18
Maltese_dog                   18
Shetland_sheepdog             18
teddy                         18
beagle                        18
Shih-Tzu                      17
Lakeland_terrier              17
Rottweiler                    17
kuvasz                        16
Italian_greyhound             16
          

In [32]:
pred.p2.value_counts()

Labrador_retriever                104
golden_retriever                   92
Cardigan                           73
Chihuahua                          44
Pomeranian                         42
Chesapeake_Bay_retriever           41
French_bulldog                     41
toy_poodle                         37
cocker_spaniel                     34
Siberian_husky                     33
miniature_poodle                   33
beagle                             28
Pembroke                           27
collie                             27
Eskimo_dog                         27
kuvasz                             26
Italian_greyhound                  22
American_Staffordshire_terrier     21
Pekinese                           21
toy_terrier                        20
chow                               20
Samoyed                            20
malinois                           20
miniature_pinscher                 20
Boston_bull                        19
Norwegian_elkhound                 19
Staffordshir

In [33]:
count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 3 columns):
tweet_id    2517 non-null int64
ret#        2517 non-null object
fav#        2517 non-null object
dtypes: int64(1), object(2)
memory usage: 59.1+ KB


### Quality


**tweet archive table**
- Missing value in expanded_urls
- Duplicate & error values in expanded_urls, repeated urls in one data entry
- There's error in dog names, example like 'a' or 'an' is not a name, also a lot of entry is None
- 0 value on rating_numerator also there's values that not 10 for rating_denominator
- Duplicate on dog types, there's serval dog have multiple properties which is incorrect
- We only want original ratings (no retweets) that have images

**predition table**
- Duplicate tweet_id column
- 2075 total entry, there's 200+ missing data entry compare to tweet table
- Dog type name convention is not the same (p1,p2,p3), there's lowercase also underscore into it

**count table**
- there's duplicate data in this dataset need to be removed

## Tidiness

- count table and prediction table can be inlcuded in the tweet archive table
- the dog types (doggo	floofer	pupper	puppo) can be moved into 1 single column since every dog only have one of these properties
- Timestamp datatype and also other column datatype need to be changed
- source in tweet archive table can be shortened iPhone,Vine,Twitter Web,TwitterDeck, help shorten the width of the column

- There's decimal in img_num column which should be integer


# Cleaning

### Missing Data
##### Define
There's missing value in expanded urls, for this column I'll using exisiting tweet_id re-construct the url, this will fix missing data and also duplicate/error data

##### Code

In [52]:
dogg_clean=dogg.copy()

In [54]:
expanded_url=[]
for item in dogg_clean['tweet_id']:
    urls='https://twitter.com/dog_rates/status/'+str(item)+'/photo/1'
    expanded_url.append(urls)

In [55]:
dogg_clean['expanded_urls']=expanded_url

##### Test

In [56]:
dogg_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2356 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [57]:
dogg_clean['expanded_urls'].value_counts()

https://twitter.com/dog_rates/status/668643542311546881/photo/1    1
https://twitter.com/dog_rates/status/668988183816871936/photo/1    1
https://twitter.com/dog_rates/status/838831947270979586/photo/1    1
https://twitter.com/dog_rates/status/673919437611909120/photo/1    1
https://twitter.com/dog_rates/status/670073503555706880/photo/1    1
https://twitter.com/dog_rates/status/704054845121142784/photo/1    1
https://twitter.com/dog_rates/status/745433870967832576/photo/1    1
https://twitter.com/dog_rates/status/889638837579907072/photo/1    1
https://twitter.com/dog_rates/status/696490539101908992/photo/1    1
https://twitter.com/dog_rates/status/675878199931371520/photo/1    1
https://twitter.com/dog_rates/status/758355060040593408/photo/1    1
https://twitter.com/dog_rates/status/712097430750289920/photo/1    1
https://twitter.com/dog_rates/status/824325613288833024/photo/1    1
https://twitter.com/dog_rates/status/775350846108426240/photo/1    1
https://twitter.com/dog_rates/stat

##### Define
We only want orignal tweet excluded retweet, I'll excluded by using retweeted_status_id

##### Code

In [58]:
dogg_clean=dogg_clean[dogg_clean['retweeted_status_id'].isnull()]

In [59]:
dogg_clean.drop('retweeted_status_id',axis=1,inplace=True)
dogg_clean.drop('retweeted_status_user_id',axis=1,inplace=True)
dogg_clean.drop('retweeted_status_timestamp',axis=1,inplace=True)


##### Test

In [60]:
dogg_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 14 columns):
tweet_id                 2175 non-null int64
in_reply_to_status_id    78 non-null float64
in_reply_to_user_id      78 non-null float64
timestamp                2175 non-null object
source                   2175 non-null object
text                     2175 non-null object
expanded_urls            2175 non-null object
rating_numerator         2175 non-null int64
rating_denominator       2175 non-null int64
name                     2175 non-null object
doggo                    2175 non-null object
floofer                  2175 non-null object
pupper                   2175 non-null object
puppo                    2175 non-null object
dtypes: float64(2), int64(3), object(9)
memory usage: 254.9+ KB


##### Define
There's error in dog names, I'll try to filter out if name didn't start with upper case letter

##### Code

In [61]:
dogg_clean.loc[dogg_clean['name'].str.islower(),'name']='None'
dogg_clean['name'].replace("O","O'Malley", inplace=True)

##### Test

In [62]:
dogg_clean['name'].value_counts()

None         784
Lucy          11
Charlie       11
Cooper        10
Oliver        10
Tucker         9
Penny          9
Sadie          8
Lola           8
Winston        8
Daisy          7
Toby           7
Bailey         6
Koda           6
Stanley        6
Bo             6
Jax            6
Oscar          6
Bella          6
Bentley        5
Louis          5
Chester        5
Rusty          5
Dave           5
Scout          5
Milo           5
Buddy          5
Leo            5
Jerry          4
Duke           4
            ... 
Cermet         1
Terrance       1
Harvey         1
Rufio          1
Damon          1
O'Malley       1
Finnegus       1
Ralphson       1
Stu            1
Hubertson      1
Jay            1
Leonidas       1
Amber          1
Ralph          1
Liam           1
Asher          1
Ralf           1
Rontu          1
Dug            1
Kallie         1
Kramer         1
Baloo          1
Kingsley       1
Anna           1
DonDon         1
Skittles       1
Dunkin         1
Stubert       

##### Define

There's 0 value on both rating_numerator also rating_denominator, for rating_numerator trying to determine if it's a true 0 or an error, for rating_denominator I'll fix it since it's always 10

##### Code

In [65]:
dogg_clean[dogg_clean['rating_numerator']==0]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
315,835152434251116546,NaN,NaN,2017-02-24 15:40:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you're so blinded by your systematic plag...,https://twitter.com/dog_rates/status/835152434...,0,10,None,None,None,None,None
1016,746906459439529985,7.468859e+17,4.196984e+09,2016-06-26 03:22:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...","PUPDATE: can't see any. Even if I could, I cou...",https://twitter.com/dog_rates/status/746906459...,0,10,None,None,None,None,None


The first row tweet_id 835152434251116546 is actually a rate 0/10 

the second row tweet_id 746906459439529985 is irrelevent about real dog rating, this entry will be removed

In [66]:
dogg_clean.drop(1016,inplace=True)

In [67]:
dogg_clean[dogg_clean['rating_denominator']!=10]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
313,835246439529840640,8.352460e+17,2.625958e+07,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@jonnysun @Lin_Manuel ok jomny I know you're e...,https://twitter.com/dog_rates/status/835246439...,960,0,None,None,None,None,None
342,832088576586297345,8.320875e+17,3.058208e+07,2017-02-16 04:45:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@docmisterio account started on 11/15/15,https://twitter.com/dog_rates/status/832088576...,11,15,None,None,None,None,None
433,820690176645140481,NaN,NaN,2017-01-15 17:52:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",The floofs have been released I repeat the flo...,https://twitter.com/dog_rates/status/820690176...,84,70,None,None,None,None,None
516,810984652412424192,NaN,NaN,2016-12-19 23:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Sam. She smiles 24/7 &amp; secretly aspir...,https://twitter.com/dog_rates/status/810984652...,24,7,Sam,None,None,None,None
902,758467244762497024,NaN,NaN,2016-07-28 01:00:57 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Why does this never happen at my front door......,https://twitter.com/dog_rates/status/758467244...,165,150,None,None,None,None,None
1068,740373189193256964,NaN,NaN,2016-06-08 02:41:38 +0000,"<a href=""http://twitter.com/download/iphone"" r...","After so many requests, this is Bretagne. She ...",https://twitter.com/dog_rates/status/740373189...,9,11,None,None,None,None,None
1120,731156023742988288,NaN,NaN,2016-05-13 16:15:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to this unbelievably well behaved sq...,https://twitter.com/dog_rates/status/731156023...,204,170,None,None,None,None,None
1165,722974582966214656,NaN,NaN,2016-04-21 02:25:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Happy 4/20 from the squad! 13/10 for all https...,https://twitter.com/dog_rates/status/722974582...,4,20,None,None,None,None,None
1202,716439118184652801,NaN,NaN,2016-04-03 01:36:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bluebert. He just saw that both #Final...,https://twitter.com/dog_rates/status/716439118...,50,50,Bluebert,None,None,None,None
1228,713900603437621249,NaN,NaN,2016-03-27 01:29:02 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Happy Saturday here's 9 puppers on a bench. 99...,https://twitter.com/dog_rates/status/713900603...,99,90,None,None,None,None,None


After checking these tweet, fixing the denominator and numerator below,drop non-rate tweet entry

In [68]:
dogg_clean.loc[dogg_clean['tweet_id']==835246439529840640,'rating_denominator']=10
dogg_clean.loc[dogg_clean['tweet_id']==835246439529840640,'rating_numerator']=13

In [69]:
dogg_clean.drop(342,inplace=True)

In [70]:
dogg_clean.drop(516,inplace=True)

In [72]:
dogg_clean.loc[dogg_clean['tweet_id']==820690176645140481,'rating_denominator']=10
dogg_clean.loc[dogg_clean['tweet_id']==820690176645140481,'rating_numerator']=12

In [73]:
dogg_clean.loc[dogg_clean['tweet_id']==758467244762497024,'rating_denominator']=10
dogg_clean.loc[dogg_clean['tweet_id']==758467244762497024,'rating_numerator']=11

In [74]:
dogg_clean.loc[dogg_clean['tweet_id']==740373189193256964,'rating_denominator']=10
dogg_clean.loc[dogg_clean['tweet_id']==740373189193256964,'rating_numerator']=14

In [75]:
dogg_clean.loc[dogg_clean['tweet_id']==731156023742988288,'rating_denominator']=10
dogg_clean.loc[dogg_clean['tweet_id']==731156023742988288,'rating_numerator']=12

In [76]:
dogg_clean.loc[dogg_clean['tweet_id']==722974582966214656,'rating_denominator']=10
dogg_clean.loc[dogg_clean['tweet_id']==722974582966214656,'rating_numerator']=13

In [77]:
dogg_clean.loc[dogg_clean['tweet_id']==716439118184652801,'rating_denominator']=10
dogg_clean.loc[dogg_clean['tweet_id']==716439118184652801,'rating_numerator']=11

In [78]:
dogg_clean.loc[dogg_clean['tweet_id']==713900603437621249,'rating_denominator']=10
dogg_clean.loc[dogg_clean['tweet_id']==713900603437621249,'rating_numerator']=11

In [79]:
dogg_clean.loc[dogg_clean['tweet_id']==710658690886586372,'rating_denominator']=10
dogg_clean.loc[dogg_clean['tweet_id']==710658690886586372,'rating_numerator']=10

In [80]:
dogg_clean.loc[dogg_clean['tweet_id']==709198395643068416,'rating_denominator']=10
dogg_clean.loc[dogg_clean['tweet_id']==709198395643068416,'rating_numerator']=9

In [81]:
dogg_clean.loc[dogg_clean['tweet_id']==704054845121142784,'rating_denominator']=10
dogg_clean.loc[dogg_clean['tweet_id']==704054845121142784,'rating_numerator']=12

In [82]:
dogg_clean.loc[dogg_clean['tweet_id']==697463031882764288,'rating_denominator']=10
dogg_clean.loc[dogg_clean['tweet_id']==697463031882764288,'rating_numerator']=11

In [83]:
dogg_clean.drop(1598,inplace=True)

In [84]:
dogg_clean.loc[dogg_clean['tweet_id']==684222868335505415,'rating_denominator']=10
dogg_clean.loc[dogg_clean['tweet_id']==684222868335505415,'rating_numerator']=11

In [85]:
dogg_clean.drop(1634,inplace=True)

In [86]:
dogg_clean.loc[dogg_clean['tweet_id']==682962037429899265,'rating_denominator']=10
dogg_clean.loc[dogg_clean['tweet_id']==682962037429899265,'rating_numerator']=10

In [87]:
dogg_clean.drop(1663,inplace=True)

In [88]:
dogg_clean.loc[dogg_clean['tweet_id']==677716515794329600,'rating_denominator']=10
dogg_clean.loc[dogg_clean['tweet_id']==677716515794329600,'rating_numerator']=12

In [89]:
dogg_clean.loc[dogg_clean['tweet_id']==675853064436391936,'rating_denominator']=10
dogg_clean.loc[dogg_clean['tweet_id']==675853064436391936,'rating_numerator']=11

In [90]:
dogg_clean.loc[dogg_clean['tweet_id']==666287406224695296,'rating_denominator']=10
dogg_clean.loc[dogg_clean['tweet_id']==666287406224695296,'rating_numerator']=9

##### Test

In [91]:
dogg_clean[dogg_clean['rating_denominator']!=10]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


##### Define

Duplicate on dog stages, there's serval dog have multiple properties which is incorrect, I'll check the actual text in the tweet and correct the dog stages

##### Code

In [92]:
dogg_clean[(dogg_clean["doggo"]=='doggo') & (dogg_clean["pupper"] == 'pupper')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
460,817777686764523521,NaN,NaN,2017-01-07 16:59:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...","This is Dido. She's playing the lead role in ""...",https://twitter.com/dog_rates/status/817777686...,13,10,Dido,doggo,None,pupper,None
531,808106460588765185,NaN,NaN,2016-12-12 00:29:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have Burke (pupper) and Dexter (doggo)...,https://twitter.com/dog_rates/status/808106460...,12,10,None,doggo,None,pupper,None
565,802265048156610565,7.331095e+17,4.196984e+09,2016-11-25 21:37:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...","Like doggo, like pupper version 2. Both 11/10 ...",https://twitter.com/dog_rates/status/802265048...,11,10,None,doggo,None,pupper,None
575,801115127852503040,NaN,NaN,2016-11-22 17:28:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bones. He's being haunted by another d...,https://twitter.com/dog_rates/status/801115127...,12,10,Bones,doggo,None,pupper,None
705,785639753186217984,NaN,NaN,2016-10-11 00:34:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Pinot. He's a sophisticated doggo. You...,https://twitter.com/dog_rates/status/785639753...,10,10,Pinot,doggo,None,pupper,None
733,781308096455073793,NaN,NaN,2016-09-29 01:42:20 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...","Pupper butt 1, Doggo 0. Both 12/10 https://t.c...",https://twitter.com/dog_rates/status/781308096...,12,10,None,doggo,None,pupper,None
889,759793422261743616,NaN,NaN,2016-07-31 16:50:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...","Meet Maggie &amp; Lila. Maggie is the doggo, L...",https://twitter.com/dog_rates/status/759793422...,12,10,Maggie,doggo,None,pupper,None
956,751583847268179968,NaN,NaN,2016-07-09 01:08:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Please stop sending it pictures that don't eve...,https://twitter.com/dog_rates/status/751583847...,5,10,None,doggo,None,pupper,None
1063,741067306818797568,NaN,NaN,2016-06-10 00:39:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is just downright precious af. 12/10 for ...,https://twitter.com/dog_rates/status/741067306...,12,10,None,doggo,None,pupper,None
1113,733109485275860992,NaN,NaN,2016-05-19 01:38:16 +0000,"<a href=""http://twitter.com/download/iphone"" r...","Like father (doggo), like son (pupper). Both 1...",https://twitter.com/dog_rates/status/733109485...,12,10,None,doggo,None,pupper,None


In [94]:
dogg_clean.loc[dogg_clean['tweet_id']==817777686764523521,'doggo']='None'
dogg_clean.loc[dogg_clean['tweet_id']==808106460588765185,'doggo']='None'
dogg_clean.loc[dogg_clean['tweet_id']==802265048156610565,'doggo']='None'
dogg_clean.loc[dogg_clean['tweet_id']==801115127852503040,'doggo']='None'
dogg_clean.loc[dogg_clean['tweet_id']==785639753186217984,'pupper']='None'
dogg_clean.loc[dogg_clean['tweet_id']==781308096455073793,'doggo']='None'
dogg_clean.loc[dogg_clean['tweet_id']==759793422261743616,'pupper']='None'
dogg_clean.loc[dogg_clean['tweet_id']==751583847268179968,'pupper']='None'
dogg_clean.loc[dogg_clean['tweet_id']==741067306818797568,'doggo']='None'
dogg_clean.loc[dogg_clean['tweet_id']==733109485275860992,'doggo']='None'


In [95]:
dogg[(dogg["doggo"]=='doggo') & (dogg["puppo"] == 'puppo')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
191,855851453814013952,NaN,NaN,2017-04-22 18:31:02 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a puppo participating in the #ScienceMa...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/855851453...,13,10,None,doggo,None,None,puppo


In [96]:
dogg_clean.loc[dogg_clean['tweet_id']==855851453814013952,'doggo']='None'



In [97]:
dogg[(dogg["doggo"]=='doggo') & (dogg["floofer"] == 'floofer')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
200,854010172552949760,NaN,NaN,2017-04-17 16:34:26 +0000,"<a href=""http://twitter.com/download/iphone"" r...","At first I thought this was a shy doggo, but i...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/854010172...,11,10,None,doggo,floofer,None,None


In [98]:
dogg_clean.loc[dogg_clean['tweet_id']==854010172552949760,'doggo']='None'


##### Test

In [99]:
(dogg_clean['doggo']+dogg_clean['floofer']+dogg_clean['pupper']+dogg_clean['puppo']).value_counts()

NoneNoneNoneNone       1825
NoneNonepupperNone      231
doggoNoneNoneNone        78
NoneNoneNonepuppo        25
NoneflooferNoneNone      10
dtype: int64

##### Define
Duplicated value in count table

##### Code

In [100]:
count=count.drop_duplicates()

##### Test

In [101]:
count.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2350 entries, 0 to 2516
Data columns (total 3 columns):
tweet_id    2350 non-null int64
ret#        2350 non-null object
fav#        2350 non-null object
dtypes: int64(1), object(2)
memory usage: 73.4+ KB



##### Define

There's lot of error non-dog type value in p1,p2,p3 column, I'll remove those non-relative value and replace underscroe with space and capitalize letter

##### Code

In [102]:
pred_clean=pred.copy()

In [103]:
pred_clean.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [104]:
pred_clean.p1=pred_clean.p1.str.replace('_',' ').str.title()

In [105]:
pred_clean.p2=pred_clean.p2.str.replace('_',' ').str.title()

In [106]:
pred_clean.p3=pred_clean.p3.str.replace('_',' ').str.title()

##### Test

In [108]:
pred_clean.p1.value_counts()

Golden Retriever             150
Labrador Retriever           100
Pembroke                      89
Chihuahua                     83
Pug                           57
Chow                          44
Samoyed                       43
Toy Poodle                    39
Pomeranian                    38
Malamute                      30
Cocker Spaniel                30
French Bulldog                26
Chesapeake Bay Retriever      23
Miniature Pinscher            23
Seat Belt                     22
German Shepherd               20
Staffordshire Bullterrier     20
Siberian Husky                20
Web Site                      19
Cardigan                      19
Shetland Sheepdog             18
Eskimo Dog                    18
Teddy                         18
Maltese Dog                   18
Beagle                        18
Rottweiler                    17
Shih-Tzu                      17
Lakeland Terrier              17
Italian Greyhound             16
Kuvasz                        16
          

##### Define

Both predition & retweet/favorites counts can be joined into one table,
also to complience with tidy data requirements, which are:

Each variable forms a column.

Each observation forms a row.

Each type of observational unit forms a table.

we are using inner join to only using 1996 entry in total.

##### Code

Now left join the other 2 table using tweet_id as key

In [109]:
comb=dogg_clean.set_index('tweet_id').join(count.set_index('tweet_id'),how='inner')

In [110]:
comb=comb.join(pred_clean.set_index('tweet_id'),how='inner')

In [112]:
comb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1996 entries, 666020888022790149 to 892420643555336193
Data columns (total 26 columns):
in_reply_to_status_id    21 non-null float64
in_reply_to_user_id      21 non-null float64
timestamp                1996 non-null object
source                   1996 non-null object
text                     1996 non-null object
expanded_urls            1996 non-null object
rating_numerator         1996 non-null int64
rating_denominator       1996 non-null int64
name                     1996 non-null object
doggo                    1996 non-null object
floofer                  1996 non-null object
pupper                   1996 non-null object
puppo                    1996 non-null object
ret#                     1996 non-null object
fav#                     1996 non-null object
jpg_url                  1996 non-null object
img_num                  1996 non-null int64
p1                       1996 non-null object
p1_conf                  1996 non-null 

### Tidiness

##### Define
Source column can be simplified by values from the text on the href, I'll shorten it as  iPhone,Vine,Twitter Web,TwitterDeck

##### Code

In [113]:
comb.loc[comb['source']=='<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>','source']='iPhone'
comb.loc[comb['source']=='<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>','source']='Vine'
comb.loc[comb['source']=='<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>','source']='Twitter Web'
comb.loc[comb['source']=='<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>','source']='TwitterDeck'


##### Test

In [114]:
comb.source.value_counts()

iPhone         1957
Twitter Web      28
TwitterDeck      11
Name: source, dtype: int64

##### Define
the dog types (doggo floofer pupper	puppo) can be moved into 1 single column since every dog only have one of these properties

##### Code

In [115]:
comb['type']='NaN'

In [116]:
comb.loc[comb['doggo']=='doggo','type']='doggo'
comb.loc[comb['floofer']=='floofer','type']='floofer'
comb.loc[comb['pupper']=='pupper','type']='pupper'
comb.loc[comb['puppo']=='puppo','type']='puppo'
comb.loc[comb['type']=='NaN','type']='None'

In [117]:
comb.drop('doggo',axis=1,inplace=True)
comb.drop('floofer',axis=1,inplace=True)
comb.drop('pupper',axis=1,inplace=True)
comb.drop('puppo',axis=1,inplace=True)

##### Test

In [118]:
comb.type.value_counts()

None       1689
pupper      209
doggo        66
puppo        24
floofer       8
Name: type, dtype: int64

In [120]:
comb.head(1)

,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,ret#,...,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,type
tweet_id,,,,,,,,,,,,,,,,,,,,,
666020888022790149,NaN,NaN,2015-11-15 22:32:08 +0000,iPhone,Here we have a Japanese Irish Setter. Lost eye...,https://twitter.com/dog_rates/status/666020888...,8,10,None,522,...,Welsh Springer Spaniel,0.465074,True,Collie,0.156665,True,Shetland Sheepdog,0.061428,True,None


##### Define

Removing column that not useful for analysis 

##### Code

In [121]:
comb.drop('in_reply_to_status_id',axis=1,inplace=True)
comb.drop('in_reply_to_user_id',axis=1,inplace=True)


##### Test

In [122]:
comb.head(1)

,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,ret#,fav#,jpg_url,...,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,type
tweet_id,,,,,,,,,,,,,,,,,,,,,
666020888022790149,2015-11-15 22:32:08 +0000,iPhone,Here we have a Japanese Irish Setter. Lost eye...,https://twitter.com/dog_rates/status/666020888...,8,10,None,522,2558,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,...,Welsh Springer Spaniel,0.465074,True,Collie,0.156665,True,Shetland Sheepdog,0.061428,True,None


##### Define

There's decimal in img_num which should be a integer number, I'll using str.strip() remove decimal

##### Code

In [124]:
comb.img_num=comb.img_num.astype(int)

##### Test

In [125]:
comb.img_num.value_counts()

1    1713
2     191
3      62
4      30
Name: img_num, dtype: int64

##### Define

Change timestamp to datetime datatype, also other item to catogory datatype

##### Code

In [126]:
comb.timestamp = pd.to_datetime(comb.timestamp)

In [128]:
comb.source = comb.source.astype('category')
comb.type = comb.type.astype('category')
comb.name = comb.name.astype('str')
comb['ret#']=comb['ret#'].astype('int')
comb['fav#']=comb['fav#'].astype('int')


In [133]:
cols=['timestamp',
 'source',
 'ret#',
 'fav#',
 'text',
 'name',
 'type',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog']

comb=comb[cols]

##### Test

In [134]:
comb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1996 entries, 666020888022790149 to 892420643555336193
Data columns (total 21 columns):
timestamp             1996 non-null datetime64[ns]
source                1996 non-null category
ret#                  1996 non-null int64
fav#                  1996 non-null int64
text                  1996 non-null object
name                  1996 non-null object
type                  1996 non-null category
expanded_urls         1996 non-null object
rating_numerator      1996 non-null int64
rating_denominator    1996 non-null int64
jpg_url               1996 non-null object
img_num               1996 non-null int64
p1                    1996 non-null object
p1_conf               1996 non-null float64
p1_dog                1996 non-null bool
p2                    1996 non-null object
p2_conf               1996 non-null float64
p2_dog                1996 non-null bool
p3                    1996 non-null object
p3_conf               1996 non-null floa

Finally, save the final dataframe to a csv file

In [135]:
comb.to_csv('twitter_archive_master.csv')

First, I would remove the column that not useful for visulization such as urls, also the original text

In [136]:
df=pd.read_csv('twitter_archive_master.csv')